# Quantitative Cryptocurrency Strategy

# Factor Selection:

## Momentum: Rate of Change (ROC) with 11-day rate for closing prices
#### ROC is a momentum based indicator, which measures the percentage change in price over the specified period to identify trends.
## Liquidity: 11-day Average Daily Volume (ADV)
#### Higher trading volume implies higher liquidity, which means that a larger number of assets can be bought or sold without causing significant change in price.
#### This is to ensure that the trades are only executed when the market shows adequate liquidity.
## Volatility: 14-day Average True Range (ATR)
#### ATR is a measure of market volatility, which captures the range within which the asset price moves over a given period. 
#### This strategy uses ATR to dynamically adjust the position sizes. When the market volatility is high, position size is reduced to mitigate the risk, and vice versa.
## --------------------------------------------------------------------
## Selected Asset: "ETH-USD" from Yahoo Finance (daily timeframe)


In [1]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

import pandas as pd
import yfinance as yf
import datetime
import numpy as np

C:\Users\emir.e\Anaconda3\lib\site-packages\backtesting\_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
class CryptoStrat(Strategy):
    # Initialization method for setting up indicators
    def init(self):
        # Calculate the 11-day Rate of Change for momentum
        self.roc = self.I(self.rate_of_change, self.data.Close, 11)
        # Calculate the 11-day Average Daily Volume for liquidity
        self.adv = self.I(self.average_daily_volume, self.data.Volume, 11)
        # Calculate the 14-day Average True Range for volatility
        self.atr = self.I(self.average_true_range, self.data.High, self.data.Low, self.data.Close, 14)

    # Static method to calculate the Rate of Change
    @staticmethod
    def rate_of_change(prices, period):
        roc = np.zeros_like(prices)
        # Compute ROC as the percentage difference from the specified period
        roc[period:] = prices[period:] / prices[:-period] - 1
        return roc

    # Static method to calculate the Average Daily Volume
    @staticmethod
    def average_daily_volume(volume, period):
        adv = np.zeros_like(volume)
        # Compute ADV as the mean volume over the specified period
        for i in range(period, len(volume)):
            adv[i] = np.mean(volume[i-period:i])
        return adv

    # Static method to calculate the Average True Range
    @staticmethod
    def average_true_range(high, low, close, period):
        high_low = high - low
        high_close = np.abs(high - np.roll(close, 1))
        low_close = np.abs(low - np.roll(close, 1))
        # Calculate the true range and then the ATR as its rolling mean
        tr = np.max((high_low, high_close, low_close), axis=0)
        atr = pd.Series(tr).rolling(period).mean()
        return atr

    # Method defining the trading logic for each step in the backtest
    def next(self):
        # Calculate the position size based on current volatility (ATR)
        position_size = self.equity / self.atr[-1] * 0.01
        # Ensure the position size is at least 1
        position_size = max(1, round(position_size))

        # Check if current ADV is above its historical median to assess liquidity
        high_liquidity = self.adv[-1] > np.median(self.adv)

        # Trading logic: buy (go long) or sell (go short) based on ROC, ensuring high liquidity
        if high_liquidity and self.roc[-1] > 0 and not self.position.is_long:
            self.position.close()
            self.buy(size=position_size)
        elif high_liquidity and self.roc[-1] < 0 and not self.position.is_short:
            self.position.close()
            self.sell(size=position_size)

# Download historical data for ETH-USD from Yahoo Finance
data = yf.download("ETH-USD", start="2018-01-01", end="2024-01-01")

# Set up and execute the backtest
bt = Backtest(data, CryptoStrat, cash=10000, commission=.002)
# Run the backtest
stats = bt.run()
# Plot the backtest results
bt.plot()
# Display the backtest statistics
print(stats)

[*********************100%***********************]  1 of 1 completed


Start                     2018-01-01 00:00:00
End                       2023-12-31 00:00:00
Duration                   2190 days 00:00:00
Exposure Time [%]                   99.315381
Equity Final [$]                 34581.679673
Equity Peak [$]                  36542.739188
Return [%]                         245.816797
Buy & Hold Return [%]              195.282184
Return (Ann.) [%]                   22.960803
Volatility (Ann.) [%]               19.883274
Sharpe Ratio                          1.15478
Sortino Ratio                        2.273751
Calmar Ratio                          1.76058
Max. Drawdown [%]                  -13.041614
Avg. Drawdown [%]                   -2.222822
Max. Drawdown Duration      510 days 00:00:00
Avg. Drawdown Duration       22 days 00:00:00
# Trades                                  173
Win Rate [%]                        39.884393
Best Trade [%]                      89.463209
Worst Trade [%]                    -59.446036
Avg. Trade [%]                    

## Comments on Strategy:
#### 5 years of daily data for ETH-USD from Yahoo Finance is utilized.
#### Strategy has generated 245.81% returns, which means it performed 50.53% better than a "Buy & Hold" approach with 195.28% return. 
#### Multiple iterations with different inputs for ROC, ADV and ATR has been backtested to generate a Sharpe Ratio of 1.15 (>1.0)
#### While a version that didn't use ADV has generated a max drawdown of about 20% with about 250 trades, incorporating ADV lowered the max drawdown to 13.04% and limited the number of trades to 173.
#### One trade in <19 Mar 2023 Sun> generated the worst drawdown with -59.44%. Further limitations in position sizes or limitation to trading in weekend can be implemented to decrease the extreme shortfalls.
